In [88]:
# The goal is to ensemble the best logistic regression model, knn classifier, and decision tree via a Hard Voting Classifier:

In [89]:
# Required import statements:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.regularizers import l2
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from google.colab import files

In [90]:
# Keras Logarithmic Regression Code From week_04 Condensed in One Cell:

def logarithmic_regression_keras(train_x, train_y, test_x, test_y):

  # Build the model:
  model = Sequential()
  model.add(Dense(10, input_shape=(2,), activation='relu', name='fc1', bias_regularizer=l2(0.01)))
  model.add(Dense(10, activation='relu', name='fc2', bias_regularizer=l2(0.01)))
  model.add(Dense(3, activation='softmax', name='output', bias_regularizer=l2(0.01)))

  # Adam optimizer with learning rate of 0.001
  optimizer = Adam(lr=0.001)
  model.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

  # Train the model
  model.fit(train_x, train_y, verbose=2, batch_size=5, epochs=200)

  # Test on unseen data
  results = model.evaluate(test_x, test_y)

  # Find the individual outputs of the method:
  ind_outputs = model.predict(test_x)
  results = []
  for flower in ind_outputs:
    if flower[0] == max(flower):
      results.append(0)
    if flower[1] == max(flower):
      results.append(1)
    if flower[2] == max(flower):
      results.append(2)
  return results

In [91]:
# knn code pasted and condensed in one cell:

# Helper method to implement Knn:
def distance(x1, y1, x2, y2):
  return np.sqrt( (x1-x2)**2 + (y1-y2)**2)

def implement_knn(k, x, y, train_x, train_y):
  closestPoints = []
  for i in range(len(train_x)):
    distancee = distance(x, y, train_x[i][0], train_x[i][1])
    closestPoints.append([distancee, train_y[i] ])
  sortedDistances = sorted(closestPoints, key=lambda x: x[0])
  # Calculate which element is closest:
  setosa = 0
  versicolor = 0
  virginica = 0
  for i in range(k):
    if sortedDistances[i][1] == 0:
      setosa += 1
    elif sortedDistances[i][1] == 1:
      versicolor += 1
    elif sortedDistances[i][1] == 2:
      virginica += 1
  if setosa >= versicolor and setosa >= virginica:
    return 0
  elif versicolor >= setosa and versicolor >= virginica:
    return 1
  else:
    return 2

# Produce output of entire array:
def knn_predictions(test_x, train_x, train_y):
  knn_predictions = []
  for i in range(len(test_x) ):
    knn_predictions.append(implement_knn(5, test_x[i][0], test_x[i][1], train_x, train_y) )
  return knn_predictions

In [92]:
# Decision Tree Code:
def dec_tree(x, y):
  tree_clf = DecisionTreeClassifier(max_depth=5) # Textbook uses 2 as a max depth.
  tree_clf.fit(x,y) # Fit data to tree
  predictions = tree_clf.predict(test_x)
  tree_predictions = []
  for i in range(len(predictions)):
    tree_predictions.append(predictions[i])
  return tree_predictions

In [93]:
# load the iris dataset: 
iris_data = load_iris()

# Set x as flower characteristics and y as target values:
x = iris_data.data[:,[2,3] ]            # Use only 2 characteristics from the iris dataset.
y = iris_data.target 

# Split the data for training and testing:
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.20)

# One hot encode some of the code for the logaritmic regression code:
train_y_log = np.eye(3)[train_y]
test_y_log = np.eye(3)[test_y]

In [86]:
# Voting classifier:
def voting_classifier():
  log_reg_results = logarithmic_regression_keras(train_x, train_y_log, test_x, test_y_log)
  knn_results = knn_predictions(test_x, train_x, train_y)
  dec_tree_results = dec_tree(test_x, test_y)
  voting_predictions= []
  for i in range(len(test_y) ):
    results = [knn_results[i], log_reg_results[i], dec_tree_results[i] ]
    if knn_results[i] == log_reg_results[i] or  knn_results[i] == dec_tree_results[i]:
      voting_predictions.append(knn_results[i])
    elif log_reg_results[i] == dec_tree_results[i]:
      voting_predictions.append(log_reg_results[i])
    else:
      # No most freq output:
      voting_predictions.append(-1)
  return voting_predictions

# Determine accuracy of voting classifier:
def voting_accuracy(voting_predictions, test_y):
  correct = 0
  for i in range(len(voting_predictions) ):
    if voting_predictions[i] == test_y[i]:
      correct += 1
  accuracy = correct/len(test_y)
  print("Voting Classifier Accuracy:", accuracy)

In [94]:
voting_predictions = voting_classifier()

Epoch 1/200
24/24 - 1s - loss: 1.2325 - accuracy: 0.3500
Epoch 2/200
24/24 - 0s - loss: 1.1452 - accuracy: 0.1417
Epoch 3/200
24/24 - 0s - loss: 1.0985 - accuracy: 0.2917
Epoch 4/200
24/24 - 0s - loss: 1.0614 - accuracy: 0.3750
Epoch 5/200
24/24 - 0s - loss: 1.0346 - accuracy: 0.5583
Epoch 6/200
24/24 - 0s - loss: 1.0111 - accuracy: 0.6333
Epoch 7/200
24/24 - 0s - loss: 0.9839 - accuracy: 0.6833
Epoch 8/200
24/24 - 0s - loss: 0.9555 - accuracy: 0.6917
Epoch 9/200
24/24 - 0s - loss: 0.9229 - accuracy: 0.6917
Epoch 10/200
24/24 - 0s - loss: 0.8937 - accuracy: 0.6917
Epoch 11/200
24/24 - 0s - loss: 0.8588 - accuracy: 0.6917
Epoch 12/200
24/24 - 0s - loss: 0.8251 - accuracy: 0.7000
Epoch 13/200
24/24 - 0s - loss: 0.7929 - accuracy: 0.7083
Epoch 14/200
24/24 - 0s - loss: 0.7586 - accuracy: 0.7167
Epoch 15/200
24/24 - 0s - loss: 0.7264 - accuracy: 0.7083
Epoch 16/200
24/24 - 0s - loss: 0.6938 - accuracy: 0.7500
Epoch 17/200
24/24 - 0s - loss: 0.6623 - accuracy: 0.7417
Epoch 18/200
24/24 - 0s

In [95]:
# Determine accuracy of voting classifier:
voting_accuracy(voting_predictions, test_y)

Voting Classifier Accuracy: 1.0
